# PkgEvalAnalysis

Latest pkgeval: https://s3.amazonaws.com/julialang-reports/nanosoldier/pkgeval/by_hash/cdf2f46_vs_d63aded/report.html

In [14]:
using DataFrames, Feather
using JuliaRegistryAnalysis
using Graphs, MetaGraphs
using Downloads

include("add_back_logs.jl")

download_logs (generic function with 1 method)

In [15]:
baseline = "cdf2f46"
vs       = "d63aded"

data_dir = joinpath("data_$(baseline)_vs_$(vs)")
if !isdir(data_dir)
    url = "https://github.com/JuliaCI/NanosoldierReports/blob/master/pkgeval/by_hash/$(baseline)_vs_$(vs)/data.tar.zst?raw=true"
    file = Downloads.download(url)
    mkdir(data_dir)
    mv(file, joinpath(data_dir, "$(baseline)_vs_$(vs).tar.zst"))
    cd(data_dir) do
        run(`tar -xf $(baseline)_vs_$(vs).tar.zst`)
    end
end

# The data in the data.tar.zst do not contain the actual logs so we have to download these
if !isfile(joinpath(data_dir, "primary_log.feather"))
    add_back_logs(data_dir)
end
primary  = Feather.read(joinpath(@__DIR__, data_dir, "primary_log.feather"));
against  = Feather.read(joinpath(@__DIR__, data_dir, "against_log.feather"));

In [16]:
# We sort packages according to how many transitive dependencies they have,
graph = JuliaRegistryAnalysis.dependency_graph(; include=(name, uuid) -> true)
name_to_vertex = Dict{String, Int}(get_prop(graph, i, :label) => i for i in 1:nv(graph));

const trans = transitiveclosure(graph.graph)

sortby = x -> begin
    v = get(name_to_vertex, x, 0)
    v == 0 && return 0
    return Graphs.indegree(trans, v)
end
    
primary = sort(primary, order(:package, by=sortby, rev=true))
against = sort(against, order(:package, by=sortby, rev=true))
nothing

In [17]:
# This is a list of different categories of failures.
# Some are common between different Julia version upgrades
# and some are "special" for this specific upgrade.

@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    UnexpectedPass
    BadBoy
    SyntaxError
    VisualRegression
    MissingDep
    BuildError
    UnsatReq
    Belapsed
    StderrCheck
    TypeInferenceError
    
    # From PkgEval
    MissingDependency
    Inactivity
    MissingBinary
    Untestable
    LogLimit
    TimeLimit
    Syntax
    
    # Specific
    
    # Assertions
    PHINodesGrouped
    CTXSSAValueAssigned
    MethodAddedWhenDeleted
    CycleDepth
    IfElseInvalidIR
    MalformedIsdefined
    BadSignatureEnzyme
    GCPreserveEnd
    NewObjMethInst
    IJLTypesEqual
    InfStackOverflow
    CodeInstMaxWorld
    SubArrayConvert

    # Segfaults
    Vload
end


In [18]:
# Join the primary and against data
package_results = leftjoin(primary, against, on=:package, makeunique=true, indicator=:source);

# Only keep packages that started to fail in the new version
fails = filter(test->test.source == "both" &&
                 test.status != test.status_1 &&
                 test.status in (":fail", ":kill", ":crash"), package_results)


missing_binary_idx     = fails.reason .== ":binary_dependency"
inactivity_idx         = fails.reason .== ":inactivity"
missing_dependency_idx = fails.reason .== ":missing_dependency"
untestable_idx         = fails.reason .== ":untestable"
log_limit_idx          = fails.reason .== ":log_limit"
time_limit_idx         = fails.reason .== ":time_limit"

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

fails.why[missing_binary_idx]     .= MissingBinary
fails.why[inactivity_idx]         .= Inactivity
fails.why[missing_dependency_idx] .= MissingDependency
fails.why[untestable_idx]         .= Untestable
fails.why[log_limit_idx]          .= LogLimit
fails.why[time_limit_idx]         .= TimeLimit

nothing

In [19]:
# Some utility functions

query(fails, s) = sort(filter(row -> occursin(s, row[:log]), fails))

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

update_reason! (generic function with 1 method)

In [21]:
# Here we pattern match certain test errors and categorize them based on that.

# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "detect_ambiguities(", TestAmbiguities)

update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "Error: doctest failure in ", DocTest)

update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, r"Expression: (\S*) ≈ (\S*)", ApproxError) 
update_reason!(fails, "Expression: ≈(", ApproxError)
    
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, " ambiguities found", NewAmbiguity)

update_reason!(fails, "Unsatisfiable requirements detected for package", UnsatReq)



update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)
# update_reason!(fails, r"Expression: (\N.*?) == ", PrintingChange)
update_reason!(fails, "Evaluated: occursin(", PrintingChange)
update_reason!(fails, "Evaluated: endswith(", PrintingChange)
update_reason!(fails, "- DIFF ------------------------", PrintingChange)
update_reason!(fails, "LoadError: syntax", SyntaxError)

update_reason!(fails, "Image did not match reference image", VisualRegression)

update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)
update_reason!(fails, "Unexpected Pass", UnexpectedPass)

update_reason!(fails, "Error building ", BuildError)
update_reason!(fails, "isempty(stderr_content)", StderrCheck)

update_reason!(fails, "isempty(stderr_content)", StderrCheck)

update_reason!(fails, "fatal error in type inference", TypeInferenceError)


# Specific ones for this release

# Assertions
update_reason!(fails, "!ctx.ssavalue_assigned.at", CTXSSAValueAssigned)
update_reason!(fails, "PHI nodes not grouped at top of basic block", PHINodesGrouped)
update_reason!(fails, "method cannot be added at the same time as method deleted", MethodAddedWhenDeleted)
update_reason!(fails, "cycle == depth", CycleDepth)
update_reason!(fails, "select i1 %ifelse", IfElseInvalidIR)
update_reason!(fails, "malformed isdefined expression", MalformedIsdefined)
update_reason!(fails, "Calling a function with bad signature", BadSignatureEnzyme)
update_reason!(fails, "llvm.julia.gc_preserve_end", GCPreserveEnd)
update_reason!(fails, "newobj == (jl_method_instance_t", NewObjMethInst)
update_reason!(fails, "!ijl_types_equal(mi->specType", IJLTypesEqual)
update_reason!(fails, "stack overflow in type inference", InfStackOverflow)
update_reason!(fails, "This might be caused by recursion over very long", InfStackOverflow)
update_reason!(fails, "(&codeinst->max_world) == WORLD_AGE_REVALIDATION_SENTINEL", CodeInstMaxWorld)

# Segfaults
update_reason!(fails, "__vload at", Vload)



nothing

In [22]:
# Packages that have gotten some attention but haven't made a new version that would
# remove the package from the PkgEval list

issues_opened = [
]

likely_tol = [
]

fixed = [
]

ignored_packages = [
]

Any[]

In [23]:
# Total package failures that we haven't categorized
total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails)) - length(issues_opened) - length(fixed)
total_unknown(fails)

931

In [24]:
# Unknown failures, these are packages where we haven't yet managed to categorize their failure

fail_unknowns = filter(x -> x.why == Unknown && 
       !(x.package in issues_opened) && 
       !(x.package in likely_tol) && 
       !(x.package in ignored_packages) && 
       !(x.package in fixed) 
      # && x.status == ":crash", 
    , fails).package

print(join(fail_unknowns[1:50], '\n'))

Requires
StaticArrays
PtrArrays
Adapt
QuadGK
PDMats
ArrayInterface
StringManipulation
InlineStrings
Colors
ExprTools
MbedTLS
ProgressMeter
CpuId
CPUSummary
SIMDTypes
MutableArithmetics
RuntimeGeneratedFunctions
ADTypes
StructArrays
SymbolicIndexingInterface
ArrayLayouts
EnzymeCore
Functors
Polyester
Transducers
LazyArrays
FastBroadcast
DiffEqBase
Krylov
SparseMatrixColorings
LoopVectorization
Sixel
Zygote
RoundingEmulator
ImageIO
Rotations
MaybeInplace
NonlinearSolveBase
DomainSets
GenericSchur
Optimisers
TiledIteration
MLUtils
ComputationalResources
OrdinaryDiffEqRKN
OrdinaryDiffEqSymplecticRK
OrdinaryDiffEqExtrapolation
SymbolicUtils
MathTeXEngine

In [25]:
# Categorize

z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    n > 0 && push!(z, (n, i))
end
sort!(z; rev=true)

14-element Vector{Any}:
 (931, Unknown)
 (580, SyntaxError)
 (236, TimeLimit)
 (10, UnsatReq)
 (9, ApproxError)
 (7, NewExceptionType)
 (6, PrintingChange)
 (5, Inactivity)
 (5, DocTest)
 (4, LogLimit)
 (4, NewAmbiguity)
 (2, InferredFailure)
 (1, GCPreserveEnd)
 (1, TestAmbiguities)

In [12]:
filter(x -> x.why == NewAmbiguity,  fails).package

String[]